In [18]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Create Overview

In [19]:
# Read the Excel file into a DataFrame, skipping row 12 and using row 11 as the header
info = pd.read_excel('pris og rækkevidde.xlsx', sheet_name='med attributes', header=10, usecols="A:K")
info = info.drop(columns=['Elektrisk rækkevidde (NEDC)'])
info = info.drop(index=0).reset_index(drop=True) # Drop row 0 (no info)

info = info.rename(columns={'Nr.':'ID', 'Mærke':'Manufacturer', 'Elektrisk rækkevidde (WLTP)': 'Range', 'Udsalgspris': 'Price', 'kW (power)': 'HP', 
                            'Ladetid (fastcharge 10-70)': 'Fast charge (min)', 'Type': 'Type', 'Segment': 'Segment', 'Country': 'Country'})

info['HP'] = (info['HP'] * 1.341)


# Display the resulting DataFrame
print(info)

              ID Manufacturer      Model        Range            Price  \
0     1.00000000       Aiways         U5 400.23038397  330825.78947368   
1     2.00000000       Aiways         U6 405.00000000  375710.00000000   
2     3.00000000         Audi     e-tron 375.78400000  714953.03016688   
3     4.00000000         Audi  e-tron GT 472.15204678 1081416.34131737   
4     5.00000000         Audi  Q4 e-tron 496.95164718  454724.62672152   
..           ...          ...        ...          ...              ...   
193 194.00000000      Xingyun        UTV  70.00000000       0.00000000   
194 195.00000000      Xinyang        ATV   0.00000000       0.00000000   
195 196.00000000        Xpeng         G9 512.62886598       0.00000000   
196 197.00000000        Xpeng         P5 450.00000000       0.00000000   
197 198.00000000        Xpeng         P7 546.84210526       0.00000000   

              HP  Fast charge (min)   Type Segment Country  
0   201.15000000        34.00000000    SUV       C

In [20]:
# Read the Excel file into a DataFrame, skipping row 12 and using row 11 as the header
total_market_share = pd.read_excel('Nyregistreringer pr. model, 01-01-2013 - 31-12-2023.xlsx', header=11, usecols="A,Z:AA")
total_market_share = total_market_share.drop(index=0).reset_index(drop=True) # Drop row 0 (no info)

# Rename columns
total_market_share = total_market_share.rename(columns={'Nr.':'ID', 'Antal.11': 'Count', 'Andel.11': 'Share'}) 

# Display share in percentage
total_market_share['Percentage'] = ((total_market_share['Share'] * 100).round(2)).astype(str)+'%'

# Display the resulting DataFrame
print(total_market_share)

              ID  Count      Share Percentage
0     1.00000000    617 0.00416416      0.42%
1     2.00000000      3 0.00002025       0.0%
2     3.00000000   1579 0.01065675      1.07%
3     4.00000000    171 0.00115409      0.12%
4     5.00000000   5646 0.03810514      3.81%
..           ...    ...        ...        ...
193 194.00000000     29 0.00019572      0.02%
194 195.00000000      4 0.00002700       0.0%
195 196.00000000    582 0.00392795      0.39%
196 197.00000000      4 0.00002700       0.0%
197 198.00000000     76 0.00051293      0.05%

[198 rows x 4 columns]


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
# Merge DataFrames based on the 'ID' column
overview = pd.merge(info, total_market_share, on='ID', how='left')

# Removes columns with no HP
overview['HP'] = overview['HP'].fillna('Nan')
overview = overview[overview['HP'] != "Nan"]

# Convert columns to integer
convert_to_int = ['ID', 'Range', 'HP', 'Fast charge (min)']
overview[convert_to_int] = overview[convert_to_int].astype(int)

# Display the resulting DataFrame
print(overview)
# Export the DataFrame to an Excel file
overview.to_excel('Overview.xlsx', index=False)

      ID Manufacturer      Model  Range            Price   HP  \
0      1       Aiways         U5    400  330825.78947368  201   
1      2       Aiways         U6    405  375710.00000000  214   
2      3         Audi     e-tron    375  714953.03016688  402   
3      4         Audi  e-tron GT    472 1081416.34131737  522   
4      5         Audi  Q4 e-tron    496  454724.62672152  281   
..   ...          ...        ...    ...              ...  ...   
186  187   Volkswagen        up!    256  173761.91926256   81   
187  188        Volvo        C40    466  449203.70797546  402   
188  189        Volvo       EX30    475  368245.00000000  268   
189  190        Volvo       XC40    457  443449.84952038  402   
191  192        Voyah       Free    500  504768.50000000  482   

     Fast charge (min)       Type Segment Country  Count      Share Percentage  
0                   34        SUV       C      CH    617 0.00416416      0.42%  
1                   34        SUV       C      CH      3 

# Descriptive statistics

In [22]:
overview[['Share', 'Range','Price', 'HP','Fast charge (min)','Type', 'Segment', 'Count']].describe()

,Share,Range,Price,HP,Fast charge (min),Count
count,112.00000000,112.00000000,112.00000000,112.00000000,112.00000000,112.00000000
mean,0.00881426,411.94642857,490635.38066830,263.56250000,34.41071429,1306.00000000
std,0.01843637,111.28300416,277620.66189183,151.62363202,12.85928947,2731.69882133
min,0.00000675,127.00000000,143848.98377282,44.00000000,16.00000000,1.00000000
25%,0.00032902,323.25000000,312881.66491126,154.00000000,27.75000000,48.75000000
50%,0.00206858,426.50000000,423702.12783784,201.00000000,31.00000000,306.50000000
75%,0.00875014,490.00000000,571829.79946652,337.50000000,40.00000000,1296.50000000
max,0.14707530,700.00000000,1715594.40816327,675.00000000,120.00000000,21792.00000000


# Create Dataset

In [23]:
# Create a list of tuples with the data
data = [(i, year) for i in range(1, 199) for year in range(2013, 2024)]

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['ID', 'year'])
print(df)


       ID  year
0       1  2013
1       1  2014
2       1  2015
3       1  2016
4       1  2017
...   ...   ...
2173  198  2019
2174  198  2020
2175  198  2021
2176  198  2022
2177  198  2023

[2178 rows x 2 columns]


In [24]:
# Read market share
market_share = pd.read_excel('andel.xlsx', sheet_name='marketshare', header=0)
market_share = market_share.iloc[:, 1:] # Removes ID column
market_share = market_share.stack().reset_index(drop=True) # Stacks columns on top of each other
print(market_share)


0      0.00000000
1      0.00000000
2      0.00000000
3      0.00000000
4      0.00000000
          ...    
2173   0.00000000
2174   0.00000000
2175   0.00008031
2176   0.00012978
2177   0.00111596
Length: 2178, dtype: float64


In [25]:
antal = pd.read_excel('andel.xlsx', sheet_name='antal',  header=0)
antal = antal.iloc[:, 1:] # Removes ID column
antal = antal.stack().reset_index(drop=True) # Stacks columns on top of each other
print(antal)

0        0
1        0
2        0
3        0
4        0
        ..
2173     0
2174     0
2175     2
2176     4
2177    70
Length: 2178, dtype: int64


In [26]:
df['Antal'] = antal[0:] 
print(df)

       ID  year  Antal
0       1  2013      0
1       1  2014      0
2       1  2015      0
3       1  2016      0
4       1  2017      0
...   ...   ...    ...
2173  198  2019      0
2174  198  2020      0
2175  198  2021      2
2176  198  2022      4
2177  198  2023     70

[2178 rows x 3 columns]


In [27]:
pd.set_option('display.float_format', '{:.8f}'.format) # Display market share with more decimals
df['Market share'] = market_share[0:] 
print(df)

       ID  year  Antal  Market share
0       1  2013      0    0.00000000
1       1  2014      0    0.00000000
2       1  2015      0    0.00000000
3       1  2016      0    0.00000000
4       1  2017      0    0.00000000
...   ...   ...    ...           ...
2173  198  2019      0    0.00000000
2174  198  2020      0    0.00000000
2175  198  2021      2    0.00008031
2176  198  2022      4    0.00012978
2177  198  2023     70    0.00111596

[2178 rows x 4 columns]


In [28]:
# Merge the original DataFrame with the prices DataFrame based on the 'ID' column
df2 = pd.merge(df, info, on='ID', how='left')
df2['HP'] = df2['HP'].fillna('Nan')
df2 = df2[df2['HP'] != "Nan"]

# Convert columns to integer
df2[convert_to_int] = df2[convert_to_int].astype(int)

# Display the resulting DataFrame
print(df2)

       ID  year  Antal  Market share Manufacturer Model  Range  \
0       1  2013      0    0.00000000       Aiways    U5    400   
1       1  2014      0    0.00000000       Aiways    U5    400   
2       1  2015      0    0.00000000       Aiways    U5    400   
3       1  2016      0    0.00000000       Aiways    U5    400   
4       1  2017      0    0.00000000       Aiways    U5    400   
...   ...   ...    ...           ...          ...   ...    ...   
2107  192  2019      0    0.00000000        Voyah  Free    500   
2108  192  2020      0    0.00000000        Voyah  Free    500   
2109  192  2021      0    0.00000000        Voyah  Free    500   
2110  192  2022      0    0.00000000        Voyah  Free    500   
2111  192  2023      8    0.00012754        Voyah  Free    500   

               Price   HP  Fast charge (min) Type Segment Country  
0    330825.78947368  201                 34  SUV       C      CH  
1    330825.78947368  201                 34  SUV       C      CH  
2  

In [29]:
# Export the DataFrame to an Excel file
df2.to_excel('Dataset.xlsx', index=False)